# OpenAI Agent

## OpenAI agent with simple calculation tools

In [1]:
from llama_index.tools import FunctionTool
from llama_index.agent import OpenAIAgent

In [2]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

In [3]:
def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b

add_tool = FunctionTool.from_defaults(fn=add)

In [5]:
agent = OpenAIAgent.from_tools([multiply_tool, add_tool], verbose=True)

In [9]:
agent.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.



Human:  Hi!


Assistant: Hello! How can I assist you today?



Human:  Can you calculate things for me?


Assistant: Of course! I can help you with calculations. What do you need to calculate?



...

Human:  multiple by 10


=== Calling Function ===
Calling function: multiply with args: {
  "a": 4,
  "b": 10
}
Got output: 40
Assistant: If you multiply 4 by 10, the result is 40.



Human:  Thanks!


Assistant: You're welcome! If you have any more calculations or any other questions, feel free to ask.



Human:  exit


## Build your own agent loop

With the new OpenAI API that supports function calling, it's never been easier to build your own agent loop.

Here's how to write your own (slightly simplified version) of the `OpenAIAgent` you see above. 

In [21]:
from llama_index.agent.utils import FunctionMessage, monkey_patch_langchain

# TODO: right now langchain does not support function messages
#       we need to monkey patch it to support it
monkey_patch_langchain()

In [47]:
import json
from typing import Sequence

from langchain.chat_models import ChatOpenAI
from langchain.memory import ChatMessageHistory
from llama_index.tools import BaseTool

In [74]:
class YourOpenAIAgent:
    def __init__(
        self,
        tools: Sequence[BaseTool] = [],
        llm: ChatOpenAI = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-next"),
        chat_history: ChatMessageHistory = ChatMessageHistory(),
    ) -> None:
        self._llm = llm
        self._tools = {tool.metadata.name: tool for tool in tools}
        self._chat_history = chat_history

    def reset(self) -> None:
        self._chat_history.clear()

    def chat(self, message: str) -> str:
        chat_history = self._chat_history
        chat_history.add_user_message(message)
        functions = [tool.metadata.to_openai_function() for _, tool in self._tools.items()]

        ai_message = self._llm.predict_messages(chat_history.messages, functions=functions)
        chat_history.add_message(ai_message)

        function_call = ai_message.additional_kwargs.get("function_call", None)
        if function_call is not None:
            function_message = self._call_function(function_call)
            chat_history.add_message(function_message)
            ai_message = self._llm.predict_messages(chat_history.messages)
            chat_history.add_message(ai_message)

        return ai_message.content

    def _call_function(self, function_call: dict) -> FunctionMessage:
        tool = self._tools[function_call["name"]]
        output = tool(**json.loads(function_call["arguments"]))
        return FunctionMessage(
            name=function_call["name"],
            content=str(output), 
        )

In [75]:
agent = YourOpenAIAgent(tools=[multiply_tool, add_tool])

In [76]:
agent.chat('Hi')

'Hello! How can I assist you today?'

In [77]:
agent.chat('What is 2 * 2')

'2 multiplied by 2 equals 4.'